# 🚨 Pitfall 01: PSM 常见失败模式与排查

---

## 🎬 真实场景：一个「成功」的 PSM 分析

小张刚学完 PSM，信心满满地分析公司的会员效果：

```python
# 小张的代码
from sklearn.neighbors import NearestNeighbors

# 估计倾向得分
lr = LogisticRegression()
lr.fit(X, T)
propensity = lr.predict_proba(X)[:, 1]

# 匹配
nn = NearestNeighbors(n_neighbors=1)
nn.fit(propensity[T==0].reshape(-1, 1))
matched_idx = nn.kneighbors(propensity[T==1].reshape(-1, 1))[1].flatten()

# 计算效应
ate = Y[T==1].mean() - Y[T==0][matched_idx].mean()
print(f"会员效应: {ate:.2f}")  # 输出: 会员效应: 50.23
```

老板很高兴：「50 块的效应！值得推广！」

**三个月后...**

推广效果远不如预期，复盘发现：

- ❌ 从没检查过匹配后的 **Balance**
- ❌ 匹配丢失了 **30% 的样本**
- ❌ 存在严重的 **共同支撑违背**

**这 50 块根本不可信！**

---

## 📚 本节学习目标

完成本练习后，你将能够：

1. 🎯 识别 PSM 分析中的 **4 种常见失败模式**
2. 🎯 使用 **SMD（标准化均值差）** 评估匹配质量
3. 🎯 正确处理 **共同支撑违背** 问题
4. 🎯 掌握 **PSM 诊断清单**，避免踩坑
5. 🎯 在面试中回答「PSM 有什么问题」

> ⚠️ **Pitfall 系列** 专注于实际工作中的常见错误，帮你避免踩坑！

In [ ]:
# 导入必要的库
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import NearestNeighbors
from scipy import stats
from typing import Tuple, Dict, List
import warnings
warnings.filterwarnings('ignore')

# 设置
plt.rcParams['font.sans-serif'] = ['Arial Unicode MS', 'SimHei']
plt.rcParams['axes.unicode_minus'] = False
sns.set_style('whitegrid')
np.random.seed(42)

print("✅ 环境准备完毕！")
print("🚨 Pitfall 系列: 今天学习 PSM 的常见失败模式")

---

## 🔬 Part 1: 生成「有问题」的数据

我们故意生成一个有各种问题的数据集，来演示 PSM 的失败模式。

In [ ]:
def generate_problematic_data(
    n: int = 5000,
    seed: int = 42
) -> Tuple[np.ndarray, np.ndarray, np.ndarray, Dict]:
    """
    生成一个「有问题」的数据集
    
    问题：
    1. 处理组和对照组的特征分布差异大（共同支撑差）
    2. 存在选择偏差
    3. 异质性效应
    """
    np.random.seed(seed)
    
    # 特征
    age = np.random.normal(40, 15, n).clip(18, 80)
    income = np.random.lognormal(10.5, 0.8, n)  # 收入，偏态分布
    education = np.random.choice([1, 2, 3, 4], n, p=[0.2, 0.3, 0.35, 0.15])  # 1=高中, 4=研究生
    
    X = np.column_stack([age, income, education])
    feature_names = ['年龄', '收入', '教育水平']
    
    # 处理分配：严重依赖收入和教育
    # 高收入、高教育更可能成为会员
    propensity_logit = (
        -3 +
        0.00002 * income +      # 收入越高越可能
        0.5 * education +       # 教育越高越可能
        0.01 * age              # 年龄有一点影响
    )
    true_propensity = 1 / (1 + np.exp(-propensity_logit))
    T = np.random.binomial(1, true_propensity)
    
    # 真实效应：异质性
    true_ate = 30  # 平均效应
    cate = (
        true_ate +
        0.0001 * income +       # 高收入效应更大
        5 * education +         # 高教育效应更大
        np.random.randn(n) * 10
    )
    
    # 结果
    baseline = (
        100 +
        0.0005 * income +
        10 * education +
        0.5 * age +
        np.random.randn(n) * 20
    )
    Y = baseline + T * cate
    
    # 计算真实效应
    true_att = cate[T == 1].mean()
    
    info = {
        'feature_names': feature_names,
        'true_att': true_att,
        'true_propensity': true_propensity,
        'cate': cate
    }
    
    print(f"📊 数据生成完成")
    print(f"=" * 50)
    print(f"样本量: {n}")
    print(f"处理组: {T.sum()} ({T.mean():.1%})")
    print(f"对照组: {(1-T).sum()} ({1-T.mean():.1%})")
    print(f"真实 ATT: {true_att:.2f}")
    
    return X, T, Y, info

In [ ]:
# 生成数据
X, T, Y, info = generate_problematic_data(n=5000)
TRUE_ATT = info['true_att']

---

## 🚨 失败模式 1: 未检查 Balance

这是**最常见**的错误：做完匹配就直接计算效应，从不检查匹配质量。

### 什么是 Balance？

Balance 指匹配后处理组和对照组的协变量分布是否相似。

### SMD（标准化均值差）

$$SMD = \frac{\bar{X}_{treated} - \bar{X}_{control}}{\sqrt{(s^2_{treated} + s^2_{control})/2}}$$

**判断标准：**
- SMD < 0.1: ✅ Balance 好
- SMD 0.1-0.2: ⚠️ 勉强接受
- SMD > 0.2: ❌ Balance 差

In [ ]:
def calculate_smd(
    X: np.ndarray,
    T: np.ndarray,
    feature_names: List[str] = None
) -> pd.DataFrame:
    """
    计算各特征的标准化均值差 (SMD)
    """
    if feature_names is None:
        feature_names = [f'X{i}' for i in range(X.shape[1])]
    
    results = []
    for i, name in enumerate(feature_names):
        treated_mean = X[T == 1, i].mean()
        control_mean = X[T == 0, i].mean()
        treated_std = X[T == 1, i].std()
        control_std = X[T == 0, i].std()
        
        pooled_std = np.sqrt((treated_std**2 + control_std**2) / 2)
        smd = (treated_mean - control_mean) / pooled_std
        
        results.append({
            '特征': name,
            '处理组均值': treated_mean,
            '对照组均值': control_mean,
            'SMD': smd,
            '|SMD|': abs(smd),
            '状态': '✅' if abs(smd) < 0.1 else ('⚠️' if abs(smd) < 0.2 else '❌')
        })
    
    return pd.DataFrame(results)

In [ ]:
# 检查匹配前的 Balance
print("📊 匹配前的 Balance 检查")
print("=" * 60)
smd_before = calculate_smd(X, T, info['feature_names'])
display(smd_before)

In [ ]:
# 可视化匹配前的分布差异
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

for i, (ax, name) in enumerate(zip(axes, info['feature_names'])):
    ax.hist(X[T==0, i], bins=30, alpha=0.5, label='对照组', color='steelblue', density=True)
    ax.hist(X[T==1, i], bins=30, alpha=0.5, label='处理组', color='coral', density=True)
    ax.set_xlabel(name, fontsize=12)
    ax.set_ylabel('密度', fontsize=12)
    ax.set_title(f'{name} 分布对比\nSMD = {smd_before["SMD"].iloc[i]:.3f}', fontsize=14)
    ax.legend()

plt.tight_layout()
plt.show()

print("\n⚠️ 问题诊断:")
print("   - 收入和教育水平的 SMD 都很大")
print("   - 处理组和对照组的分布差异明显")
print("   - 如果不解决这个问题，效应估计会有偏")

### 「错误」的 PSM 分析（不检查 Balance）

In [ ]:
def bad_psm_analysis(X, T, Y):
    """
    错误的 PSM 分析：不检查 Balance
    """
    # 估计倾向得分
    lr = LogisticRegression(max_iter=1000)
    lr.fit(X, T)
    propensity = lr.predict_proba(X)[:, 1]
    
    # 最近邻匹配
    treated_ps = propensity[T == 1].reshape(-1, 1)
    control_ps = propensity[T == 0].reshape(-1, 1)
    
    nn = NearestNeighbors(n_neighbors=1)
    nn.fit(control_ps)
    distances, matched_idx = nn.kneighbors(treated_ps)
    matched_idx = matched_idx.flatten()
    
    # 直接计算效应（错误做法！）
    Y_treated = Y[T == 1]
    Y_control_matched = Y[T == 0][matched_idx]
    ate = Y_treated.mean() - Y_control_matched.mean()
    
    return {
        'ate': ate,
        'propensity': propensity,
        'matched_idx': matched_idx,
        'distances': distances.flatten()
    }

# 运行错误的分析
bad_result = bad_psm_analysis(X, T, Y)

print("❌ 错误的 PSM 分析（不检查 Balance）")
print("=" * 50)
print(f"真实 ATT: {TRUE_ATT:.2f}")
print(f"估计 ATT: {bad_result['ate']:.2f}")
print(f"偏差: {bad_result['ate'] - TRUE_ATT:+.2f}")
print("\n⚠️ 这个结果看起来「正常」，但我们还没检查 Balance！")

In [ ]:
# 检查匹配后的 Balance
treated_idx = np.where(T == 1)[0]
control_idx = np.where(T == 0)[0][bad_result['matched_idx']]

X_matched = np.vstack([X[treated_idx], X[control_idx]])
T_matched = np.array([1] * len(treated_idx) + [0] * len(control_idx))

print("📊 匹配后的 Balance 检查")
print("=" * 60)
smd_after = calculate_smd(X_matched, T_matched, info['feature_names'])
display(smd_after)

print("\n💡 观察:")
print("   - SMD 有所改善，但仍有问题")
print("   - 这说明匹配质量不够好")

In [ ]:
# Love Plot: 可视化 SMD 变化
def love_plot(smd_before, smd_after, feature_names):
    fig, ax = plt.subplots(figsize=(10, 6))
    
    y_pos = np.arange(len(feature_names))
    
    # 匹配前
    ax.scatter(smd_before['|SMD|'], y_pos, s=100, marker='o', 
              label='匹配前', color='red', alpha=0.7)
    
    # 匹配后
    ax.scatter(smd_after['|SMD|'], y_pos, s=100, marker='s', 
              label='匹配后', color='green', alpha=0.7)
    
    # 连线
    for i in range(len(feature_names)):
        ax.plot([smd_before['|SMD|'].iloc[i], smd_after['|SMD|'].iloc[i]], 
               [y_pos[i], y_pos[i]], 'gray', linestyle='--', alpha=0.5)
    
    # 参考线
    ax.axvline(0.1, color='green', linestyle='--', linewidth=2, label='SMD=0.1')
    ax.axvline(0.2, color='orange', linestyle='--', linewidth=2, label='SMD=0.2')
    
    ax.set_yticks(y_pos)
    ax.set_yticklabels(feature_names)
    ax.set_xlabel('|SMD|', fontsize=12)
    ax.set_title('Love Plot: 匹配前后 SMD 对比', fontsize=14)
    ax.legend(loc='upper right')
    ax.set_xlim([0, max(smd_before['|SMD|'].max() * 1.1, 0.5)])
    
    plt.tight_layout()
    plt.show()

love_plot(smd_before, smd_after, info['feature_names'])

---

## 🚨 失败模式 2: 共同支撑违背

**共同支撑 (Common Support / Overlap)**：处理组和对照组的倾向得分分布应该有足够的重叠。

如果某些处理组样本在对照组中找不到「相似」的匹配对象，强行匹配会导致偏差。

In [ ]:
# 可视化共同支撑
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 倾向得分分布
propensity = bad_result['propensity']
axes[0].hist(propensity[T==0], bins=50, alpha=0.5, label='对照组', color='steelblue', density=True)
axes[0].hist(propensity[T==1], bins=50, alpha=0.5, label='处理组', color='coral', density=True)
axes[0].set_xlabel('倾向得分', fontsize=12)
axes[0].set_ylabel('密度', fontsize=12)
axes[0].set_title('倾向得分分布', fontsize=14)
axes[0].legend()

# 共同支撑区域
ps_min_treated = propensity[T==1].min()
ps_max_treated = propensity[T==1].max()
ps_min_control = propensity[T==0].min()
ps_max_control = propensity[T==0].max()

common_min = max(ps_min_treated, ps_min_control)
common_max = min(ps_max_treated, ps_max_control)

axes[1].hist(propensity[T==0], bins=50, alpha=0.5, label='对照组', color='steelblue', density=True)
axes[1].hist(propensity[T==1], bins=50, alpha=0.5, label='处理组', color='coral', density=True)
axes[1].axvline(common_min, color='green', linestyle='--', linewidth=2)
axes[1].axvline(common_max, color='green', linestyle='--', linewidth=2)
axes[1].axvspan(common_min, common_max, alpha=0.2, color='green', label='共同支撑区域')
axes[1].set_xlabel('倾向得分', fontsize=12)
axes[1].set_ylabel('密度', fontsize=12)
axes[1].set_title('共同支撑区域', fontsize=14)
axes[1].legend()

plt.tight_layout()
plt.show()

# 统计违背情况
n_treated_outside = ((propensity[T==1] < common_min) | (propensity[T==1] > common_max)).sum()
n_control_outside = ((propensity[T==0] < common_min) | (propensity[T==0] > common_max)).sum()

print(f"📊 共同支撑检查")
print(f"   共同支撑区域: [{common_min:.4f}, {common_max:.4f}]")
print(f"   处理组在区域外: {n_treated_outside} ({n_treated_outside/T.sum():.1%})")
print(f"   对照组在区域外: {n_control_outside} ({n_control_outside/(1-T).sum():.1%})")

In [ ]:
# 检查匹配距离
print("📊 匹配距离统计")
print("=" * 50)
print(f"最小距离: {bad_result['distances'].min():.6f}")
print(f"最大距离: {bad_result['distances'].max():.6f}")
print(f"均值: {bad_result['distances'].mean():.6f}")
print(f"中位数: {np.median(bad_result['distances']):.6f}")

# 大距离匹配的比例
large_distance_pct = (bad_result['distances'] > 0.05).mean()
print(f"\n距离 > 0.05 的比例: {large_distance_pct:.1%}")

if large_distance_pct > 0.1:
    print("\n⚠️ 警告: 有较多匹配的距离很大，质量存疑！")

---

## 🚨 失败模式 3: 样本丢失过多

当使用 caliper（卡尺）约束时，如果丢失了太多样本，会导致：
1. 估计的目标人群改变
2. 结果的外推性下降

In [ ]:
def psm_with_caliper(
    X: np.ndarray,
    T: np.ndarray,
    Y: np.ndarray,
    caliper: float = 0.2
) -> Dict:
    """
    带卡尺约束的 PSM
    
    caliper: 倾向得分标准差的倍数
    """
    # 估计倾向得分
    lr = LogisticRegression(max_iter=1000)
    lr.fit(X, T)
    propensity = lr.predict_proba(X)[:, 1]
    
    # 计算卡尺阈值
    ps_std = propensity.std()
    caliper_threshold = caliper * ps_std
    
    # 匹配
    treated_ps = propensity[T == 1]
    control_ps = propensity[T == 0]
    treated_idx = np.where(T == 1)[0]
    control_idx_all = np.where(T == 0)[0]
    
    matched_treated = []
    matched_control = []
    used_control = set()
    
    for i, (t_idx, t_ps) in enumerate(zip(treated_idx, treated_ps)):
        # 找最近的对照组
        distances = np.abs(control_ps - t_ps)
        
        # 排除已用过的
        for used in used_control:
            distances[np.where(control_idx_all == used)[0]] = np.inf
        
        min_idx = np.argmin(distances)
        min_dist = distances[min_idx]
        
        if min_dist <= caliper_threshold:
            matched_treated.append(t_idx)
            matched_control.append(control_idx_all[min_idx])
            used_control.add(control_idx_all[min_idx])
    
    # 计算效应
    if len(matched_treated) == 0:
        return {'ate': np.nan, 'n_matched': 0, 'n_original': T.sum()}
    
    Y_treated = Y[matched_treated]
    Y_control = Y[matched_control]
    ate = Y_treated.mean() - Y_control.mean()
    
    return {
        'ate': ate,
        'n_matched': len(matched_treated),
        'n_original': T.sum(),
        'match_rate': len(matched_treated) / T.sum(),
        'caliper_threshold': caliper_threshold,
        'matched_treated_idx': matched_treated,
        'matched_control_idx': matched_control
    }

In [ ]:
# 对比不同 caliper 的效果
calipers = [0.05, 0.1, 0.2, 0.5, 1.0, None]
results = []

for caliper in calipers:
    if caliper is None:
        result = bad_psm_analysis(X, T, Y)
        result['n_matched'] = T.sum()
        result['match_rate'] = 1.0
        caliper_str = '无限制'
    else:
        result = psm_with_caliper(X, T, Y, caliper)
        caliper_str = f'{caliper:.2f}σ'
    
    results.append({
        'Caliper': caliper_str,
        '匹配样本': result['n_matched'],
        '匹配率': result.get('match_rate', 1.0),
        'ATE': result['ate'],
        '偏差': result['ate'] - TRUE_ATT
    })

results_df = pd.DataFrame(results)
print("📊 不同 Caliper 的效果对比")
print("=" * 60)
print(f"真实 ATT: {TRUE_ATT:.2f}")
print()
display(results_df)

In [ ]:
# 可视化
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 匹配率 vs Caliper
caliper_numeric = [0.05, 0.1, 0.2, 0.5, 1.0, 1.5]
match_rates = results_df['匹配率'].tolist()
axes[0].plot(caliper_numeric, match_rates, 'bo-', markersize=10, linewidth=2)
axes[0].axhline(0.9, color='green', linestyle='--', label='90% 匹配率')
axes[0].axhline(0.7, color='orange', linestyle='--', label='70% 匹配率')
axes[0].set_xlabel('Caliper (σ 的倍数)', fontsize=12)
axes[0].set_ylabel('匹配率', fontsize=12)
axes[0].set_title('Caliper 对匹配率的影响', fontsize=14)
axes[0].legend()
axes[0].set_ylim([0, 1.05])

# 偏差 vs Caliper
biases = results_df['偏差'].tolist()
axes[1].plot(caliper_numeric, biases, 'ro-', markersize=10, linewidth=2)
axes[1].axhline(0, color='black', linestyle='-', linewidth=1)
axes[1].set_xlabel('Caliper (σ 的倍数)', fontsize=12)
axes[1].set_ylabel('偏差 (估计 - 真实)', fontsize=12)
axes[1].set_title('Caliper 对偏差的影响', fontsize=14)

plt.tight_layout()
plt.show()

print("\n💡 关键发现:")
print("   1. Caliper 太紧 → 匹配率低 → 样本丢失")
print("   2. Caliper 太松 → 匹配质量差 → 偏差大")
print("   3. 需要在两者间权衡")

---

## 🚨 失败模式 4: 隐变量遗漏

即使 Balance 检查通过，如果存在**未观测的混淆变量**，估计仍然有偏。

这是 PSM（以及所有观测数据方法）的根本局限。

In [ ]:
# 模拟遗漏变量的情况

def generate_data_with_hidden_confounder(n=5000, seed=42):
    """
    生成有隐变量的数据
    
    隐变量 U 同时影响 T 和 Y，但我们观测不到
    """
    np.random.seed(seed)
    
    # 观测特征
    X1 = np.random.randn(n)
    X2 = np.random.randn(n)
    X = np.column_stack([X1, X2])
    
    # 隐变量 U（未观测）
    U = np.random.randn(n)
    
    # 处理分配（依赖 X 和 U）
    propensity_logit = 0.5 * X1 + 0.3 * X2 + 0.8 * U  # U 影响 T！
    propensity = 1 / (1 + np.exp(-propensity_logit))
    T = np.random.binomial(1, propensity)
    
    # 结果（依赖 X、T 和 U）
    true_ate = 5.0
    Y = 10 + true_ate * T + 2 * X1 + X2 + 3 * U + np.random.randn(n)  # U 也影响 Y！
    
    return X, T, Y, U, true_ate

# 生成数据
X_hidden, T_hidden, Y_hidden, U, true_ate_hidden = generate_data_with_hidden_confounder()

# 用观测变量做 PSM
lr = LogisticRegression()
lr.fit(X_hidden, T_hidden)
ps_hidden = lr.predict_proba(X_hidden)[:, 1]

# 匹配
nn = NearestNeighbors(n_neighbors=1)
nn.fit(ps_hidden[T_hidden==0].reshape(-1, 1))
matched_idx = nn.kneighbors(ps_hidden[T_hidden==1].reshape(-1, 1))[1].flatten()

ate_psm = Y_hidden[T_hidden==1].mean() - Y_hidden[T_hidden==0][matched_idx].mean()

print("📊 遗漏变量的影响")
print("=" * 50)
print(f"真实 ATE: {true_ate_hidden:.2f}")
print(f"PSM 估计 (忽略 U): {ate_psm:.2f}")
print(f"偏差: {ate_psm - true_ate_hidden:+.2f}")
print("\n⚠️ 即使 Balance 检查可能通过，估计仍然有偏！")
print("   因为 U 是未观测的混淆变量")

---

## ✅ Part 2: 正确的 PSM 诊断流程

In [ ]:
def complete_psm_analysis(
    X: np.ndarray,
    T: np.ndarray,
    Y: np.ndarray,
    feature_names: List[str] = None,
    caliper: float = 0.2,
    verbose: bool = True
) -> Dict:
    """
    完整的 PSM 分析流程
    
    包括所有必要的诊断步骤
    """
    if feature_names is None:
        feature_names = [f'X{i}' for i in range(X.shape[1])]
    
    results = {}
    
    # Step 1: 匹配前 Balance 检查
    if verbose:
        print("=" * 60)
        print("Step 1: 匹配前 Balance 检查")
        print("=" * 60)
    
    smd_before = calculate_smd(X, T, feature_names)
    results['smd_before'] = smd_before
    
    if verbose:
        display(smd_before[['特征', '|SMD|', '状态']])
        max_smd_before = smd_before['|SMD|'].max()
        if max_smd_before > 0.2:
            print(f"\n⚠️ 最大 SMD = {max_smd_before:.3f} > 0.2，Balance 较差")
    
    # Step 2: 估计倾向得分
    if verbose:
        print("\n" + "=" * 60)
        print("Step 2: 估计倾向得分")
        print("=" * 60)
    
    lr = LogisticRegression(max_iter=1000)
    lr.fit(X, T)
    propensity = lr.predict_proba(X)[:, 1]
    results['propensity'] = propensity
    
    if verbose:
        print(f"倾向得分范围: [{propensity.min():.4f}, {propensity.max():.4f}]")
        print(f"倾向得分均值: {propensity.mean():.4f}")
    
    # Step 3: 共同支撑检查
    if verbose:
        print("\n" + "=" * 60)
        print("Step 3: 共同支撑检查")
        print("=" * 60)
    
    common_min = max(propensity[T==1].min(), propensity[T==0].min())
    common_max = min(propensity[T==1].max(), propensity[T==0].max())
    results['common_support'] = (common_min, common_max)
    
    in_support = (propensity >= common_min) & (propensity <= common_max)
    pct_in_support = in_support.mean()
    results['pct_in_support'] = pct_in_support
    
    if verbose:
        print(f"共同支撑区域: [{common_min:.4f}, {common_max:.4f}]")
        print(f"在共同支撑内的样本: {pct_in_support:.1%}")
        if pct_in_support < 0.9:
            print("\n⚠️ 共同支撑覆盖率 < 90%，存在外推风险")
    
    # Step 4: 执行匹配
    if verbose:
        print("\n" + "=" * 60)
        print(f"Step 4: 执行匹配 (caliper = {caliper}σ)")
        print("=" * 60)
    
    match_result = psm_with_caliper(X, T, Y, caliper)
    results['match_result'] = match_result
    
    if verbose:
        print(f"原始处理组: {T.sum()}")
        print(f"匹配成功: {match_result['n_matched']} ({match_result['match_rate']:.1%})")
        if match_result['match_rate'] < 0.8:
            print("\n⚠️ 匹配率 < 80%，可能需要放宽 caliper")
    
    # Step 5: 匹配后 Balance 检查
    if verbose:
        print("\n" + "=" * 60)
        print("Step 5: 匹配后 Balance 检查")
        print("=" * 60)
    
    if match_result['n_matched'] > 0:
        X_matched = np.vstack([
            X[match_result['matched_treated_idx']],
            X[match_result['matched_control_idx']]
        ])
        T_matched = np.array([1] * match_result['n_matched'] + [0] * match_result['n_matched'])
        smd_after = calculate_smd(X_matched, T_matched, feature_names)
        results['smd_after'] = smd_after
        
        if verbose:
            display(smd_after[['特征', '|SMD|', '状态']])
            max_smd_after = smd_after['|SMD|'].max()
            if max_smd_after > 0.1:
                print(f"\n⚠️ 匹配后最大 SMD = {max_smd_after:.3f} > 0.1，Balance 仍不够好")
            else:
                print(f"\n✅ 所有特征 |SMD| < 0.1，Balance 良好")
    
    # Step 6: 计算效应
    if verbose:
        print("\n" + "=" * 60)
        print("Step 6: 估计效应")
        print("=" * 60)
    
    results['ate'] = match_result['ate']
    
    if verbose:
        print(f"ATT 估计: {match_result['ate']:.2f}")
    
    return results

In [ ]:
# 运行完整的 PSM 分析
complete_result = complete_psm_analysis(X, T, Y, info['feature_names'], caliper=0.2)

---

## 📋 PSM 诊断清单

In [ ]:
print("""
📋 PSM 诊断清单
================================================================================

在报告 PSM 结果之前，必须检查以下内容：

□ 1. 倾向得分模型拟合
   - 模型是否收敛？
   - AUC 是否合理（不要太高也不要太低）？
   
□ 2. 共同支撑检查
   - 两组倾向得分分布是否有足够重叠？
   - 多少样本在共同支撑区域内？
   
□ 3. 匹配质量（最重要！）
   - 所有特征的 |SMD| < 0.1？
   - 使用 Love Plot 可视化
   
□ 4. 匹配率
   - 匹配成功率是多少？
   - 丢失的样本有什么特点？
   
□ 5. 敏感性分析
   - 改变 caliper 结果是否稳定？
   - 改变匹配方法结果是否稳定？
   
□ 6. 隐变量考虑
   - 是否可能存在未观测的混淆变量？
   - 如果有，做敏感性分析（如 E-value）

================================================================================
""")

---

## 💡 思考题

### 问题 1: SMD 的阈值 0.1 是怎么来的？有没有更好的判断标准？

**你的答案:**

---

### 问题 2: 如果 Balance 检查不通过，有哪些可能的解决方案？

**你的答案:**

---

### 问题 3: PSM 丢失了很多样本，这意味着什么？对结果解释有什么影响？

**你的答案:**

---

### 问题 4: 在面试中，如果面试官问「PSM 有什么局限性」，你怎么回答？

**你的答案:**

---

---

## 📋 总结

### PSM 四大失败模式

| 失败模式 | 症状 | 诊断方法 | 解决方案 |
|---------|------|---------|----------|
| **未检查 Balance** | 估计偏差大 | 计算 SMD，绘制 Love Plot | 调整匹配方法，增加协变量 |
| **共同支撑差** | 极端权重/丢失样本 | 检查倾向得分分布重叠 | Trimming，改用其他方法 |
| **样本丢失过多** | 目标人群改变 | 检查匹配率 | 放宽 caliper，多对一匹配 |
| **隐变量遗漏** | 即使 Balance 好也有偏差 | 无法直接检测 | 敏感性分析，寻找 IV |

### 关键指标

- **SMD (标准化均值差)**: |SMD| < 0.1 为好
- **共同支撑覆盖率**: > 90% 为好
- **匹配率**: > 80% 为好

### 最佳实践

1. **先诊断，后估计**: 永远先检查 Balance
2. **可视化是必须的**: Love Plot、倾向得分分布图
3. **敏感性分析**: 改变参数看结果稳定性
4. **报告透明**: 把诊断结果也报告出来

---

**「不检查 Balance 的 PSM，就像不系安全带开车——看起来没问题，直到出事。」**

🎉 恭喜完成 Pitfall 01！